In [7]:
import json
import os
import pandas as pd
import numpy as np

def write_csv():
    with open('movielens.csv', 'w') as fp:
        fp.write(ratings.to_csv())

# Load entities
entity_labels = dict()
entity_names = dict()
with open('../data/mindreader/entities_clean.json', 'r') as file:
    data = json.load(file)
    
    for uri, name, labels in data:
        entity_labels[uri] = set(labels.split('|'))
        entity_names[uri] = name

# Load MovieLens ratings
DATA_PATH = '../data'
ml_path = os.path.join(DATA_PATH, 'movielens')

# Load from CSV
movies = pd.read_csv(f'{ml_path}/movies.csv')
ratings = pd.read_csv(f'{ml_path}/ratings.csv')
links = pd.read_csv(f'{ml_path}/links.csv')
mapping = pd.read_csv(f'{ml_path}/mapping.csv')

# Restrict mapping to URIs available in the KG 
uris = set(entity_names.keys())
mapping = mapping[mapping.uri.isin(uris)]

# Merge with mappings
movies = movies.merge(links, on='movieId')
movies.imdbId = movies.imdbId.map(lambda item: f'tt{str(item).zfill(7)}')
movies = movies.merge(mapping, on='imdbId')

# Merge ratings
ratings_before = len(ratings)
ratings = ratings.merge(movies, on='movieId')
ratings['isItem'] = True
print(f'Ratings removed: {ratings_before - len(ratings)}')

# Drop stuff we don't need
ratings.drop(["movieId", "genres", "tmdbId", "imdbId", "timestamp"], axis=1, inplace=True)
ratings.sort_values(by='userId', inplace=True)

def split(sub_matrix):
    mask = sub_matrix[np.random.random() < 0.8]
    
    return len(sub_matrix[mask]), len(sub_matrix[~mask])


index_item = dict(enumerate(ratings['uri'].cat.categories))


# Split into train and test
print(len(ratings))
ratings.groupby('userId').apply(split)

Ratings removed: 7751


AttributeError: Can only use .cat accessor with a 'category' dtype